In [ ]:
import numpy as np
import plotly.graph_objects as go

# Spiral parameters
# - center
# - number of rings
# - radius of each ring
# - height of each ring
# - number of points in each ring

center = np.array([0, 0, 0])
num_rings = 3
radii = [6, 8, 10]
heights = [10, 15, 20]
num_points = [8, 8, 8]

poses = []
orientations = []

for i in range(num_rings):
    r = radii[i]
    h = heights[i]
    n = num_points[i]

    for j in range(n):
        angle = 2*np.pi*j/n
        x = r*np.cos(angle)
        y = r*np.sin(angle)
        z = h
        poses.append([x, y, z])

        # Compute orientation
        v = [-x, -y, -z]  # Vector pointing towards the center
        v /= np.linalg.norm(v)
        v = np.array(v)
        v = v/np.linalg.norm(v)
        z = [0, 0, 1]
        x = np.cross(z, v)
        x = x/np.linalg.norm(x)
        y = np.cross(v, x)
        y = y/np.linalg.norm(y)
        

poses = np.array(poses)

fig = go.Figure(data=[go.Scatter3d(x=poses[:, 0], y=poses[:, 1], z=poses[:, 2], mode='markers')])
fig.show()

In [ ]:
def vec_to_quat(v):
    x, y, z = v
    angle = np.linalg.norm(v)
    axis = v/angle
    w = np.cos(angle/2)
    x = x*np.sin(angle/2)
    y = y*np.sin(angle/2)
    z = z*np.sin(angle/2)
    return np.array([w, x, y, z])

v = np.array([1, 0, 0])
q = vec_to_quat(v)
q

In [ ]:
import numpy as np

# Function to find the Y and Z vectors for camera orientation
def find_camera_orientation(v):
    # Normalize the look direction vector
    v = v / np.linalg.norm(v)

    # If the look direction vector is aligned with the Y-axis
    if np.allclose(v, [0, 1, 0]):
        # Choose an arbitrary vector perpendicular to the Y-axis
        y_axis = np.array([1, 0, 0])
    else:
        # Set the Y-axis vector to be orthogonal to the look direction vector
        # Choose one of the standard basis vectors (e.g., (0, 1, 0))
        y_axis = np.array([0, 1, 0])  # You can choose [1, 0, 0] as well
        y_axis = y_axis - np.dot(y_axis, v) * v
        y_axis /= np.linalg.norm(y_axis)

    # Set the Z-axis vector to be orthogonal to both the look direction vector and the Y-axis vector
    z_axis = np.cross(v, y_axis)
    z_axis /= np.linalg.norm(z_axis)
    
    return y_axis, z_axis

# Example vector representing look direction
v = np.array([0, 0.9, 0.])

# Find camera orientation vectors
y_axis, z_axis = find_camera_orientation(v)

print("Y-axis vector:", y_axis)
print("Z-axis vector:", z_axis)

In [ ]:
def rot_mat_from_vecs(u, v):
    """Compute Rotation Matrix which aligns vector u with v

    https://math.stackexchange.com/a/2672702

    Parameters
    ----------
    u : np.array (n_dim x 1)
        Vector to rotate
    v : np.array (n_dim x 1)
        Vector to align with

    Returns
    -------
    np.array (n_dim x n_dim)
        Rotation matrix

    """
    u = u.reshape(-1, 1)
    v = v.reshape(-1, 1)
    # Make sure u and v are normalized
    u /= np.linalg.norm(u)
    v /= np.linalg.norm(v)
    ndim = len(u)
    w = u + v
    return 2 * w @ w.T / (w.T @ w) - np.eye(ndim)  # Rodrigues's rotation formula

In [ ]:
rot_mat_from_vecs(np.array([1., 0., 0.]), np.array([0., 1., 0.]))

In [ ]:
import numpy as np

def rotation_matrix_from_vectors(u, v):
    # Normalize input vectors
    u_norm = u / np.linalg.norm(u)
    v_norm = v / np.linalg.norm(v)

    # Compute the rotation axis and angle between u and v
    axis = np.cross(u_norm, v_norm)
    angle = np.arccos(np.clip(np.dot(u_norm, v_norm), -1.0, 1.0))

    # Rodrigues' rotation formula
    K = np.array([[0, -axis[2], axis[1]],
                  [axis[2], 0, -axis[0]],
                  [-axis[1], axis[0], 0]])

    rotation_matrix = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)

    return rotation_matrix

# Example vectors u and v
u = np.array([1, 0, 0])
v = np.array([0, 1, 0])

# Compute rotation matrix to align u with v
R = rotation_matrix_from_vectors(u, v)

print("Rotation Matrix:")
print(R)